In [1]:
import numpy as np

# Numerical Solution of 1D Heat Equation 

(This is warm-up for callable bond pricing.)

We aim to solve
$$\begin{gather}
u_t = a u_xx \\
 x\in [0,L] \\
 t\in [0,T] \\
 u(0,x) = f(x) \\
 u(t,0) = l(x) \\
 u(t,L) = r(x)
\end{gather}$$

We'll do this via the Crank-Nicolson discretization scheme. 

## Index Notation

$$
  u_{i,j} = u(i\Delta_t, j\Delta_x),\qquad i=1,\dots,n,\, j=1,\dots,m.
$$
Since $n\Delta_t=T$ we have $\Delta_t=T/n$. Simlarly $\Delta_x=L/m$.

## Discretization
1. Time derivative is centered midway between $i$ and $i+1$:
$$
   u_t\bigr|_{((i+0.5)\Delta_t, j\Delta_x)} = \frac{u_{i+1,j} - u_{i,j}}{\Delta_t} + O(\Delta_t^2)
$$
1. Space derivative is average of the second-order differences at $i$ and $i+1$:
$$
  u_{xx}\bigr|_{((i+0.5)\Delta_t, j\Delta_x)} = \frac 12\left[ 
      \frac {u_{i,j+1} - 2u_{i,j} + u_{i,j-1} }{\Delta_x^2} + \frac {u_{i+1,j+1} - 2u_{i+1,j} + u_{i+1,j-1}}{\Delta_x^2} 
  \right] + O(\Delta_x^2)
$$
1. Dropping the error terms and putting into the PDE gives:
$$
   \frac{u_{i+1,j} - u_{i,j}}{\Delta_t} = \frac a2\left[ 
      \frac {u_{i,j+1} - 2u_{i,j} + u_{i,j-1} }{\Delta_x^2} + \frac {u_{i+1,j+1} - 2u_{i+1,j} + u_{i+1,j-1}}{\Delta_x^2} 
  \right]
$$
1. Let $\gamma=\frac a2 \frac {\Delta_t}{\Delta_x^2}$:
$$
u_{i+1,j} - u_{i,j} = \gamma\left[ 
      u_{i,j+1} - 2u_{i,j} + u_{i,j-1}  + u_{i+1,j+1} - 2u_{i+1,j} + u_{i+1,j-1} 
  \right]
$$
1. Now collect the $i+1$ terms on the LHS:
$$
u_{i+1,j} -\gamma\left[u_{i+1,j+1} - 2u_{i+1,j} + u_{i+1,j-1}\right]  = u_{i,j} + \gamma\left[ 
      u_{i,j+1} - 2u_{i,j} + u_{i,j-1}   
  \right]
$$
1. Collecting futher on the $u$ terms:
$$
 -\gamma u_{i+1,j+1} + (1+2\gamma)u_{i+1,j} -\gamma u_{i+1,j-1} = 
  \gamma u_{i,j+1} + (1-2\gamma)u_{i,j} + \gamma u_{i,j-1} 
$$

In [6]:
a = 1
L = 1
T = 1
n = 10
m = 10
dt = T/n
dx = L/m
gam = 0.5*a*dt/dx**2
print(gam)

4.999999999999999
